In [1]:
%matplotlib notebook
import torch
import numpy as np

In [2]:
# Load checkpoint
import os
from dgcnn_segmentation import Net
from SharpfData import Sharpf
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader

transform = T.Compose([
    T.RandomTranslate(0.01),
    T.RandomRotate(15, axis=0),
    T.RandomRotate(15, axis=1),
    T.RandomRotate(15, axis=2)
])
# pre_transform = T.NormalizeScale()

path = os.path.join(os.getcwd(), '..', 'data', 'abc')

test_dataset = Sharpf(path, train=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False,
                         num_workers=0)

MODEL_PATH = os.path.join(os.getcwd(), 'checkpoints', 'dgcnn-sharpf-2019-08-28.pth')
model = Net(2, k=30)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()
model.cpu()

Net(
  (conv1): DynamicEdgeConv(nn=Sequential(
    (0): Sequential(
      (0): Linear(in_features=6, out_features=64, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  ), k=30)
  (conv2): DynamicEdgeConv(nn=Sequential(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  ), k=30)
  (conv3): DynamicEdgeConv(nn=Sequential(
    (0): Sequential(
      (0): Linear(in_features=128,

In [3]:
def test(loader, model, num_samples=1):
    correct_nodes = total_nodes = 0
    pred_list = []
    data_list = []
    counter = 0
    for data in loader:
#         data = data.to(device)
        data_list.append(data)
        print('appended data')
        with torch.no_grad():
            out = model(data)
        pred = out.max(dim=1)[1]
        pred_list.append(pred)
        correct_nodes += pred.eq(data.y).sum().item()
        print('correct_nodes: {}'.format(correct_nodes))
        total_nodes += data.num_nodes
        counter += 1
        print(counter)
        print(num_samples)
        if counter == num_samples:
            break
    
    print('Accuracy: {:.4f}'.format(correct_nodes/total_nodes))
    return pred_list, data_list

In [20]:
pred_x, pc_x = test(test_loader, model, 10)

appended data
correct_nodes: 1021
1
10
appended data
correct_nodes: 2005
2
10
appended data
correct_nodes: 3002
3
10
appended data
correct_nodes: 4004
4
10
appended data
correct_nodes: 5016
5
10
appended data
correct_nodes: 6035
6
10
appended data
correct_nodes: 7059
7
10
appended data
correct_nodes: 8082
8
10
appended data
correct_nodes: 9096
9
10
appended data
correct_nodes: 10120
10
10
Accuracy: 0.9883


In [10]:
pred_x[1]

tensor([1, 1, 0,  ..., 0, 0, 0])

In [12]:
pc_x[0]

Batch(batch=[1024], pos=[1024, 3], y=[1024])

In [13]:
pc_x[0].pos

tensor([[ 0.0951, -0.0568, -0.2719],
        [-0.2642, -0.0568, -0.2719],
        [ 0.2455, -0.6961, -0.2719],
        ...,
        [-0.0855,  0.1228,  0.8409],
        [-0.2557, -0.0023, -0.0764],
        [-0.2641, -0.0512,  0.0091]])

In [14]:
pc_x[0].y

tensor([1, 1, 1,  ..., 0, 0, 0])

In [30]:
pred_n = pred_x[9]
pc_n = pc_x[9].pos
y_n = pc_x[9].y

In [31]:
type(y_n)

torch.Tensor

In [32]:
pc_n = pc_n.numpy()

In [33]:
idx = np.where(pred_n==y_n)[0]
colors = [(255,0,0,0) if i in idx and y_n[i] == 1 else (0,0,255,0) if i in idx and y_n[i] == 0\
          else (0,255,0,0) for i in range(1024)] 
# RGBA -> Red (255,0,0,0), G(0,255,0,0), blue (0,0,255,0)

In [27]:
len(colors)

1024

In [28]:
pc_n

array([[-0.47752383,  0.05503589, -0.67743033],
       [-0.47752383,  0.05503589, -0.6183118 ],
       [-0.47752383,  0.05503589, -0.5591933 ],
       ...,
       [-0.2934351 , -0.06332564, -0.67743033],
       [-0.21395591, -0.093405  , -0.67743033],
       [-0.25565517, -0.03654787, -0.67743033]], dtype=float32)

# Use trimesh for ploting

In [ ]:
import trimesh
import pyglet
import pywavefront
import matplotlib


In [ ]:
pc_tm = trimesh.points.PointCloud(vertices=pc_1, colors=colors)

In [ ]:
pc_tm.show()

In [ ]:
pc_tm.vertices

In [ ]:
vv = [[ 0.09512696, -0.05680472, -0.27188173],[-0.26416698, -0.05680472, -0.27188173]]
cl = [[255,0,0,0],[0,255,0,0]]
testpc = trimesh.points.PointCloud(vertices=vv, colors=cl)
testpc.show()

In [ ]:
testpc.vertices

In [ ]:
pc_tm.scene().show('notebook')

In [ ]:
tm.points.plot_points(pc_1)

# Use matplotlib for ploting

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# colors = ['o' if i in idx and y_1[i] == 1 else '^' if i in idx and y_1[i] == 0\
#           else "X" for i in range(1024)] 
colors = ['red' if i in idx and y_1[i] == 1 else 'blue' if i in idx and y_1[i] == 0\
          else 'green' for i in range(1024)]

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

for m, p in zip(colors, pc_1):
    xs = p[0]
    ys = p[1]
    zs = p[2]
    ax.scatter(xs, ys, zs, c=m)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

# Use plotly for ploting

In [34]:
import plotly.graph_objects as go

colors = ['red' if i in idx and y_n[i] == 1 else 'blue' if i in idx and y_n[i] == 0\
          else 'green' for i in range(1024)]
fig = go.Figure(data=[go.Scatter3d(x=pc_n[:,0], y=pc_n[:,1], z=pc_n[:,2],
                                   mode='markers',
                                   marker=dict(size=2,
                                              color=colors))])
fig.show()

In [ ]:
import ipyvolume as ipv

ipv.figure()
s = ipv.scatter(pc_1[:,0], pc_1[:,1], pc_1[:,2], marker='sphere', color=colors, size=2)
ipv.show()

In [35]:
def find_n_least_acc(loader, model, num_samples=1):
    correct_nodes = total_nodes = 0
    pred_list = []
    acc_list = []
    data_list = []
    for i, data in enumerate(loader):
        with torch.no_grad():
            out = model(data)
        pred = out.max(dim=1)[1]
        
        per_sample_correct = pred.eq(data.y).sum().item()
        correct_nodes += per_sample_correct
        per_sample_acc = per_sample_correct/data.num_nodes
        print('Sample {} accuracy: {}\r'.format(i, per_sample_acc), end="")
        total_nodes += data.num_nodes

        pred_list.append(pred)
        acc_list.append(per_sample_acc)
        data_list.append(data)
            
    idx = np.argsort(np.array(acc_list))
    
    pred_list = [pred_list[i] for i in idx[:num_samples]]
    data_list = [data_list[i] for i in idx[:num_samples]]
    acc_list = [acc_list[i] for i in idx[:num_samples]]
    print('Accuracy: {:.4f}'.format(correct_nodes/total_nodes))
    return pred_list, data_list, acc_list

In [36]:
pred, pc, acc = find_n_least_acc(test_loader, model, 5)

Accuracy: 0.9923racy: 1.0951171875


In [37]:
len(pred), len(pc), len(acc)

(5, 5, 5)

In [38]:
import plotly.graph_objects as go

pred_0 = pred[0]
pc_0 = pc[0].pos
y_0 = pc[0].y
acc_0 = acc[0]

pc_0 = pc_0.numpy()
idx = np.where(pred_0==y_0)[0]
colors_0 = ['red' if i in idx and y_0[i] == 1 else 'blue' if i in idx and y_0[i] == 0\
          else 'green' for i in range(1024)]

print('Accuracy: {}'.format(acc_0))
fig = go.Figure(data=[go.Scatter3d(x=pc_0[:,0], y=pc_0[:,1], z=pc_0[:,2],
                                   mode='markers',
                                   marker=dict(size=2,
                                              color=colors_0))])
fig.show()

Accuracy: 0.72265625


In [39]:
pred_1 = pred[1]
pc_1 = pc[1].pos
y_1 = pc[1].y
acc_1 = acc[1]

pc_1 = pc_1.numpy()
idx = np.where(pred_1==y_1)[0]
colors_1 = ['red' if i in idx and y_1[i] == 1 else 'blue' if i in idx and y_1[i] == 0\
          else 'green' for i in range(1024)]

print('Accuracy: {}'.format(acc_1))
fig = go.Figure(data=[go.Scatter3d(x=pc_1[:,0], y=pc_1[:,1], z=pc_1[:,2],
                                   mode='markers',
                                   marker=dict(size=2,
                                              color=colors_1))])
fig.show()

Accuracy: 0.78125


In [40]:
pred_2 = pred[2]
pc_2 = pc[2].pos
y_2 = pc[2].y
acc_2 = acc[2]

pc_2 = pc_2.numpy()
idx = np.where(pred_2==y_2)[0]
colors_2 = ['red' if i in idx and y_2[i] == 1 else 'blue' if i in idx and y_2[i] == 0\
          else 'green' for i in range(1024)]

print('Accuracy: {}'.format(acc_2))
fig = go.Figure(data=[go.Scatter3d(x=pc_2[:,0], y=pc_2[:,1], z=pc_2[:,2],
                                   mode='markers',
                                   marker=dict(size=2,
                                              color=colors_2))])
fig.show()

Accuracy: 0.7900390625


In [41]:
pred_3 = pred[3]
pc_3 = pc[3].pos
y_3 = pc[3].y
acc_3 = acc[3]

pc_3 = pc_3.numpy()
idx = np.where(pred_3==y_3)[0]
colors_3 = ['red' if i in idx and y_3[i] == 1 else 'blue' if i in idx and y_3[i] == 0\
          else 'green' for i in range(1024)]

print('Accuracy: {}'.format(acc_3))
fig = go.Figure(data=[go.Scatter3d(x=pc_3[:,0], y=pc_3[:,1], z=pc_3[:,2],
                                   mode='markers',
                                   marker=dict(size=2,
                                              color=colors_3))])
fig.show()

Accuracy: 0.7958984375


In [42]:
pred_4 = pred[4]
pc_4 = pc[4].pos
y_4 = pc[4].y
acc_4 = acc[4]

pc_4 = pc_4.numpy()
idx = np.where(pred_4==y_4)[0]
colors_4 = ['red' if i in idx and y_4[i] == 1 else 'blue' if i in idx and y_4[i] == 0\
          else 'green' for i in range(1024)]

print('Accuracy: {}'.format(acc_4))
fig = go.Figure(data=[go.Scatter3d(x=pc_4[:,0], y=pc_4[:,1], z=pc_4[:,2],
                                   mode='markers',
                                   marker=dict(size=2,
                                              color=colors_4))])
fig.show()

Accuracy: 0.9091796875
